In [21]:
pip install pandas transformers


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

df_ytb = pd.read_csv("/content/drive/MyDrive/CSV FILES/ytb_Pooh Tries To Cheer Up Eeyore.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(54, 2)


In [24]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

[{'label': '2', 'score': 0.7972832918167114},
 {'label': '2', 'score': 0.25198718905448914},
 {'label': '0', 'score': 0.6396825909614563},
 {'label': '1', 'score': 0.3823336064815521},
 {'label': '2', 'score': 0.7818618416786194},
 {'label': '2', 'score': 0.6471675634384155},
 {'label': '2', 'score': 0.7005420327186584},
 {'label': '2', 'score': 0.8866654634475708},
 {'label': '2', 'score': 0.9269297122955322},
 {'label': '2', 'score': 0.9164174795150757},
 {'label': '1', 'score': 0.38006073236465454},
 {'label': '1', 'score': 0.6976498961448669},
 {'label': '0', 'score': 0.8611220717430115},
 {'label': '1', 'score': 0.48428910970687866},
 {'label': '0', 'score': 0.5775516033172607},
 {'label': '2', 'score': 0.5061205625534058},
 {'label': '2', 'score': 0.6248171329498291},
 {'label': '2', 'score': 0.3756239116191864},
 {'label': '2', 'score': 0.6621161699295044},
 {'label': '1', 'score': 0.6626569032669067},
 {'label': '3', 'score': 0.47033777832984924},
 {'label': '2', 'score': 0.800

In [25]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,I Like eeyore’s tail,2,0.797283
1,Freakin hell pooh,2,0.251987
2,Poor eyeore. Getting honey shoved down his thr...,0,0.639683
3,"Poor Eeyore, he was never the same after the d...",1,0.382334
4,Wasn&#39;t this because they forgot his birthd...,2,0.781862
5,Won&#39;t they get diabetic after eating all t...,2,0.647168
6,I can&#39;t believe it! I have been spelling E...,2,0.700542
7,"Eeyore, I don’t know if this will make happy, ...",2,0.886665
8,I miss watching Winnie the Pooh when I was you...,2,0.926930
9,Eeyore you are loved by everyone❤,2,0.916417


In [26]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         2           love
1         2           love
2         0        sadness
3         1            joy
4         2           love
5         2           love
6         2           love
7         2           love
8         2           love
9         2           love
10        1            joy
11        1            joy
12        0        sadness
13        1            joy
14        0        sadness
15        2           love
16        2           love
17        2           love
18        2           love
19        1            joy
20        3          anger
21        2           love
22        2           love
23        2           love
24        2           love
25        2           love
26        2           love
27        2           love
28        1            joy
29        2           love
30        2           love
31        2           love
32        2           love
33        2           love
34        0        sadness
35        0        sadness
3

<ipython-input-26-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [27]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_Pooh Tries To Cheer Up Eeyore_with_sentiments.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,I Like eeyore’s tail,2,0.797283,love
1,Freakin hell pooh,2,0.251987,love
2,Poor eyeore. Getting honey shoved down his thr...,0,0.639683,sadness
3,"Poor Eeyore, he was never the same after the d...",1,0.382334,joy
4,Wasn&#39;t this because they forgot his birthd...,2,0.781862,love
5,Won&#39;t they get diabetic after eating all t...,2,0.647168,love
6,I can&#39;t believe it! I have been spelling E...,2,0.700542,love
7,"Eeyore, I don’t know if this will make happy, ...",2,0.886665,love
8,I miss watching Winnie the Pooh when I was you...,2,0.926930,love
9,Eeyore you are loved by everyone❤,2,0.916417,love
